In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Dataset Preprocessing**

In [3]:
targets = pd.read_csv('targets.csv')
q1sales = pd.read_csv('q1sales.csv')
productsales = pd.read_csv('q1productsales.csv')
sales = pd.read_csv('cleaned/sales.csv')

In [4]:
sales['Location'] = sales['Location'].str.replace(r'CONNECT POINT|CONNECT\s*/\s*LITE| CONNECT LITE', '', regex=True)
sales['Location'] = sales['Location'].str.strip()
#save to csv
sales.to_csv('cleaned/sales.csv', index=False)

In [5]:
# Clean up the datasets
# targets.dropna(inplace=True)
# q1sales.dropna(inplace=True)
# productsales.dropna(inplace=True)

In [6]:
targets.head(5)
q1sales.head(5)
productsales.head(5)

,LOCATION,,LOGICALS,SIM CARD,PHONES,VTU,MOMO,DEVICE TOTAL,AIRTIME
0,ALAKAHIA,"48,340,300","32,639,000","1,218,700","41,500",0,"9,126,100","5,315,000","32,639,000"
1,OWERRI 2,"28,515,200","13,095,000","1,614,900","639,000","5,909,300","3,477,000","3,780,000","19,004,300"
2,ONITSHA 2,"26,856,400","18,804,800","1,334,200",0,"292,500","3,429,900","2,995,000","19,097,300"
3,AWKA,"38,549,890","22,070,050","2,377,200","1,451,000","4,916,690","3,614,950","4,120,000","26,986,740"
4,AIRPORT,"4,852,300","970,000","216,300","76,000",0,"2,710,000","880,000","970,000"


In [7]:
targets.shape
q1sales.shape
productsales.shape

(26, 9)

In [8]:
targets.dtypes
q1sales.dtypes
productsales.dtypes

LOCATION                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [9]:
# Remove commas from the 'TOTAL' column in q1sales
# It is not a str value, but a float value with commas
# Convert the 'TOTAL' column to string first, then remove commas
q1sales['TOTAL'] = q1sales['TOTAL'].astype(str)
q1sales['TOTAL'] = q1sales['TOTAL'].str.replace(',', '')
q1sales['TOTAL'] = q1sales['TOTAL'].astype('int64')
q1sales.head(10)

,LOCATION,JANUARY,FERUARY,MARCH,TOTAL
0,ALAKAHIA,"15,807,900","16,996,800","15,535,600",48340300
1,OWERRI 2,"3,796,400","6,920,000","17,798,800",28515200
2,ONITSHA 2,"5,542,400","9,901,800","11,412,200",26856400
3,AWKA,"14,604,650","12,630,540","11,314,700",38549890
4,AIRPORT,"1,083,900","1,369,400","2,399,000",4852300
5,OKIGWE HUB,"1,541,150","1,896,050","1,636,750",5073950
6,MBAITOLI,"1,148,500","1,148,500","1,555,800",3852800
7,IHIALA,"817,600","1,092,475","1,345,325",3255400
8,IMSU,"951,550","1,327,300","1,517,200",3796050
9,ALVAN 1,"705,100","1,220,550","1,152,250",3077900


In [10]:
# First, clean and convert 'Total Paid' column if not already done
sales['Total Paid'] = sales['Total Paid'].astype(str).str.replace(',', '')
sales['Total Paid'] = pd.to_numeric(sales['Total Paid'], errors='coerce')

# Create a 'TOTAL' column in sales (you can name it differently if needed)
sales['TOTAL'] = sales['Total Paid']  # or use 'Total Amount' if that's what you need

# Clean and convert target columns
targets['DEVICE TOTAL'] = targets['DEVICE TOTAL'].astype(str).str.replace(',', '').astype('int64')
targets['AIRTIME'] = targets['AIRTIME'].astype(str).str.replace(',', '').astype('int64')
targets['MOMO'] = targets['MOMO'].astype(str).str.replace(',', '').astype('int64')

# Calculate TOTAL and 4-month target
targets['TOTAL'] = targets['DEVICE TOTAL'] + targets['AIRTIME'] + targets['MOMO']
targets['JAN - APRIL 25 TARGET'] = targets['TOTAL'] * 4

# Average sales target for Q1
avg_target = targets['JAN - APRIL 25 TARGET'].mean()

# Available locations
available_targets = targets['LOCATION'].unique()
print(f"Available locations: {available_targets}")

# Q1 sales summary for those locations
avg_sales = sales[sales['Location'].isin(available_targets)]['TOTAL'].mean()
sum_sales = sales[sales['Location'].isin(available_targets)]['TOTAL'].sum()
sum_targets = targets[targets['LOCATION'].isin(available_targets)]['JAN - APRIL 25 TARGET'].sum()

# Print result
print(f"Q1 Average Sales: NGN {avg_sales:,.0f}")
print(f"Q1 Total Sales: NGN {sum_sales:,.0f}")
print(f"Q1 Target Total: NGN {sum_targets:,.0f}")


Available locations: ['AWKA' 'OKIGWE HUB' 'OWERRI 2' 'ONITSHA 2' 'ALAKAHIA' 'PHIA' 'ALVAN 2'
 'UMUAGWO']
Q1 Average Sales: NGN 95,057
Q1 Total Sales: NGN 208,365,840
Q1 Target Total: NGN 617,739,324


In [11]:
# Calculate the percentage of sales against the target
targets['PERCENTAGE'] = (sum_sales / sum_targets) * 100
targets['PERCENTAGE'] = targets['PERCENTAGE'].round(2)


# Wrong calculation happened for ALAKAHIA 
# Manually calculate the percentage for ALAKAHIA
# Find the index of ALAKAHIA in targets
# index = targets[targets['LOCATION'] == 'ALAKAHIA'].index[0]
# # Calculate the percentage for ALAKAHIA
# targets.at[index, 'PERCENTAGE'] = (q1sales[q1sales['LOCATION'] == 'ALAKAHIA']['TOTAL'].values[0] / targets.at[index, 'JAN - MARCH 25 TARGET']) * 100
# # Round the percentage to 2 decimal places
# targets.at[index, 'PERCENTAGE'] = round(targets.at[index, 'PERCENTAGE'], 2)
# Check the updated percentage for ALAKAHIA
# print(f"ALAKAHIA percentage: {targets.at[index, 'PERCENTAGE']}%")

print(f"Q1 Average percentage of target met: {targets['PERCENTAGE'].mean():,.2f}%")
targets.head()

Q1 Average percentage of target met: 33.73%


,LOCATION,AIRTIME,MOMO,4G ROUTER,5G ROUTER,MIFI,DEVICE TOTAL,TOTAL,JAN - APRIL 25 TARGET,PERCENTAGE
0,AWKA,15500000,5500000,11660000,480000,4875000,17015000,38015000,152060000,33.73
1,OKIGWE HUB,8250000,2328000,4000000,2640000,2625000,9265000,19843000,79372000,33.73
2,OWERRI 2,14500000,6000000,1800000,800000,1350000,3950000,24450000,97800000,33.73
3,ONITSHA 2,16000000,6000000,2200000,800000,1500000,4500000,26500000,106000000,33.73
4,ALAKAHIA,11785620,8865558,2400000,1040000,1725000,5165000,25816178,103264712,33.73


In [12]:
# TOTAL TARGET FOR Q1
total_target = targets['JAN - APRIL 25 TARGET'].sum()
print(f"Q1 Total target: NGN {total_target:,.0f}")

# total target FOR EACH PRODUCT
products = ['AIRTIME', 'MOMO', 'DEVICE TOTAL']



for product in products:
    total_product_target = targets[product].sum()
    print(f"Q1 Total target for {product}: NGN {total_product_target:,.0f}")




Q1 Total target: NGN 617,739,324
Q1 Total target for AIRTIME: NGN 76,481,273
Q1 Total target for MOMO: NGN 31,693,558
Q1 Total target for DEVICE TOTAL: NGN 46,260,000


In [13]:
# Save the cleaned dataframes to new CSV file
targets.to_csv('cleaned/targets.csv', index=False)
q1sales.to_csv('cleaned/q1sales.csv', index=False)
productsales.to_csv('cleaned/productsales.csv', index=False)
sales.to_csv('cleaned/sales.csv', index=False)

In [14]:


# Prices 
# 4G router = 20,000 naira
# 4G Mifi = 15,000 naira
# 5G router = 80,000 naira


In [15]:
# Targets are unrealistic
# TOTAL,"53,202,130","64,226,265","75,082,275","192,510,670", Jan, Feb, Mar, Total

In [16]:
# Questions to answer in these datasets
# 1. What is the average sales target for Q1? - Checked
# 2. What is the actual sales for Q1? - Checked
# 3. What is the average sales for Q1? - Checked
# What percentage of the target was achieved for Q1? - Checked
# What percentage of the target was achieved for each branch? 

# Visualised Questions with Matplotlib and Seaborn
# Average percentage met for each branch 
# What percentage was achieved for each product? 
# What is the average sales for each month? 
# Worst performing shops for Q1 